In [38]:
import logging
import numpy as np
from gensim import utils
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

In [41]:
model = Word2Vec.load_word2vec_format('./models/bothwiki_lowercase_400.model', binary=False)

INFO:gensim.models.word2vec:loading projection weights from ./models/bothwiki_lowercase_400.model
INFO:gensim.models.word2vec:loaded (1152863, 400) matrix from ./models/bothwiki_lowercase_400.model
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [34]:
count = 0
correct = np.zeros(3)

questions = 'questions-translations-medium.txt'

print "Dutch to English"
for line_no, line in enumerate(utils.smart_open(questions)):
    #if count % 1000 == 0:
    #    print "#questsions seen: %s, Accuracy @1, 5, 10: %s" % (count, correct)
        
    count = count+1

    a, b, c, expected = [word.lower() for word in line.split()]
    most_similar = model.most_similar(positive=[b, c], negative=[a], topn=10)
    #most_similar = model.most_similar_cosmul(positive=[b, c], negative=[a], topn=10)

    answers = zip(*most_similar)[0]

    if expected == most_similar[0][0]:
        correct[0] = correct[0] + 1
        correct[1] = correct[1] + 1
        correct[2] = correct[2] + 1
    elif expected in answers[:5]:
        correct[1] = correct[1] + 1
        correct[2] = correct[2] + 1
    elif expected in answers:
        correct[2] = correct[2] + 1
        
print "**FINAL** Training Size: %s, Accuracy @1, 5, 10: %s" % (count, correct)

Dutch to English
**FINAL** Training Size: 19420, Accuracy @1, 5, 10: [  833.  1748.  2250.]


In [35]:
count = 0
correct = np.zeros(3)

questions = 'questions-translations-medium.txt'

print "English to Dutch"
for line_no, line in enumerate(utils.smart_open(questions)):
    #if count % 1000 == 0:
    #    print "#questsions seen: %s, Accuracy @1, 5, 10: %s" % (count, correct)
        
    count = count+1

    b, a, expected, c = [word.lower() for word in line.split()]
    most_similar = model.most_similar(positive=[b, c], negative=[a], topn=10)
    #most_similar = model.most_similar_cosmul(positive=[b, c], negative=[a], topn=10)

    answers = zip(*most_similar)[0]

    if expected == most_similar[0][0]:
        correct[0] = correct[0] + 1
        correct[1] = correct[1] + 1
        correct[2] = correct[2] + 1
    elif expected in answers[:5]:
        correct[1] = correct[1] + 1
        correct[2] = correct[2] + 1
    elif expected in answers:
        correct[2] = correct[2] + 1
        
print "**FINAL** Training Size: %s, Accuracy @1, 5, 10: %s" % (count, correct)

English to Dutch
**FINAL** Training Size: 19420, Accuracy @1, 5, 10: [  749.  1550.  2015.]


In [ ]:
import random

num_words = 100; # Number of "input" translation pairs
num_tests_per_word = 500; # Number of translations we're gonna do with this word

translations_pairs_file = 'translation_pairs.txt'
translations = []
for line_no, line in enumerate(utils.smart_open(translations_pairs_file)):
    translations.append(line)
    
print "Dutch to English - statistics on best/worst/std in base pair"
for i in range(num_words):
    # Shuffle the translations, so we test with random words each time
    shuffled = random.shuffle(translations)
    
    # Get the training relationship (this is what we take as knowledge to "train" on)
    a, b = [word.lower() for word in translations[0].split()]
    
    # Reset the scores
    correct = np.zeros(3)
    
    # Test 100 translations
    for j in range(num_tests_per_word):
        c, expected = [word.lower() for word in translations[j + 1].split()]
        try:
            most_similar = model.most_similar(positive=[b, c], negative=[a], topn=10)
            #most_similar = model.most_similar_cosmul(positive=[b, c], negative=[a], topn=10)

            answers = zip(*most_similar)[0]

            if expected == most_similar[0][0]:
                correct[0] = correct[0] + 1
                correct[1] = correct[1] + 1
                correct[2] = correct[2] + 1
            elif expected in answers[:5]:
                correct[1] = correct[1] + 1
                correct[2] = correct[2] + 1
            elif expected in answers:
                correct[2] = correct[2] + 1
        except KeyError:
            print "Not in dictionary: %s" % (line)
            
    print "Round %s/%s (%s), tested %s, correct: %s" % (i, num_words, translations[0], num_tests_per_word, correct)

Dutch to English - statistics on best/worst/std in base pair
Round 0/100 (sluiten close
), tested 500, correct: [ 13.  33.  57.]
Not in dictionary: zwijg silent
Round 1/100 (zin sentence
), tested 500, correct: [  0.   5.  13.]
Round 2/100 (he hey
), tested 500, correct: [ 1.  1.  1.]
Round 3/100 (moed courage
), tested 500, correct: [ 12.  18.  27.]
Not in dictionary: zwijg silent
Round 4/100 (hopen hope
), tested 500, correct: [  8.  37.  53.]
Not in dictionary: zwijg silent
Round 5/100 (schat treasure
), tested 500, correct: [ 18.  64.  81.]
Not in dictionary: zwijg silent
Round 6/100 (hoogte altitude
), tested 500, correct: [ 10.  29.  37.]
Round 7/100 (bereikt reached
), tested 500, correct: [  8.  19.  26.]
Not in dictionary: zwijg silent
Round 8/100 (leerde learned
), tested 500, correct: [  0.  16.  36.]
Round 9/100 (kleren clothes
), tested 500, correct: [ 12.  38.  52.]
Round 10/100 (ochtend morning
), tested 500, correct: [  6.  33.  51.]
Round 11/100 (pardon pardon
), teste

In [ ]:
import random

num_words = 100; # Number of "input" translation pairs
num_tests_per_word = 500; # Number of translations we're gonna do with this word

translations_pairs_file = 'translation_pairs.txt'
translations = []
for line_no, line in enumerate(utils.smart_open(translations_pairs_file)):
    translations.append(line)
    
print "English to Dutch - statistics on best/worst/std in base pair"
for i in range(num_words):
    # Shuffle the translations, so we test with random words each time
    shuffled = random.shuffle(translations)
    
    # Get the training relationship (this is what we take as knowledge to "train" on)
    b, a = [word.lower() for word in translations[0].split()]
    
    # Reset the scores
    correct = np.zeros(3)
    
    # Test 100 translations
    for j in range(num_tests_per_word):
        expected, c = [word.lower() for word in translations[j + 1].split()]
        
        try:
            most_similar = model.most_similar(positive=[b, c], negative=[a], topn=10)
            #most_similar = model.most_similar_cosmul(positive=[b, c], negative=[a], topn=10)

            answers = zip(*most_similar)[0]

            if expected == most_similar[0][0]:
                correct[0] = correct[0] + 1
                correct[1] = correct[1] + 1
                correct[2] = correct[2] + 1
            elif expected in answers[:5]:
                correct[1] = correct[1] + 1
                correct[2] = correct[2] + 1
            elif expected in answers:
                correct[2] = correct[2] + 1
        except KeyError:
            print "Not in dictionary: %s" % (line)
            
    print "Round %s/%s (%s), tested %s, correct: %s" % (i, num_words, translations[0], num_tests_per_word, correct)